# 1. Setup

This is a notebook in which we are taking the best-fit T_jump of HCN (h13cn, really) and deriving a binding energy for it.

$$T_{jump} = 71.6_{-2.9}^{+3.0}$$

$$X_{in} (H^{13}CN) = \left( 8.63_{-0.82}^{+0.97} \right) \times 10^{-10}$$


In [1]:
import astropy.units as u
import astropy.constants as c

X_HCN = 5.96e-8
T_jump = 71.2 * u.K
n_H2_at_Tjump = 8.226e+07

From Hollenbach et al. (2009):

 $$ T_{f,i} \simeq (E_i/k) \left[57 +
   \ln \left[ 
       \left(
           \frac{N_{s,i}}{10^{15}~\mathrm{cm}^{-2}}
           \right)
       \left( 
           \frac{\nu_i}{10^{13}~\mathrm{s}^{-1}}
           \right)
       \left(
           \frac{1~\mathrm{cm}^{-3}}{n_i}
           \right)
       \left(
           \frac{10^{4}~\mathrm{cm~s}^{-1}}{v_i}
           \right)
           \right] \right] ^{-1}
           $$
           
$$ \Rightarrow \frac{E_i}{k} = T_{f,i} \times \left[57 +
   \ln \left[ 
       \left(
           \frac{N_{s,i}}{10^{15}~\mathrm{cm}^{-2}}
           \right)
       \left( 
           \frac{\nu_i}{10^{13}~\mathrm{s}^{-1}}
           \right)
       \left(
           \frac{1~\mathrm{cm}^{-3}}{n_i}
           \right)
       \left(
           \frac{10^{4}~\mathrm{cm~s}^{-1}}{v_i}
           \right)
           \right] \right] $$

In [2]:
import numpy as np

def binding_energy_given_desorption_temp(T_fi, N_si=1e15, nu_i=1e13, n_i=1, v_i=1e4):
    
    log_arg = (N_si/1e15)*(nu_i/1e13)*(1/n_i)*(1e4/v_i)
    E_i_over_k = T_fi * (57 + np.log(log_arg))
    
    return E_i_over_k

# Sanity test - H2O
n_H2O = 2.13e8 * 5e-6
H2O_binding_energy = binding_energy_given_desorption_temp(100*u.K, n_i=n_H2O)
print("H2O binding energy derived from this formula: {:.1f}".format(H2O_binding_energy))
print("Reference H2O binding energy: 4800 K")

H2O binding energy derived from this formula: 5002.9 K
Reference H2O binding energy: 4800 K


In [3]:
HCN_binding_energy_naive = binding_energy_given_desorption_temp(T_jump)
print("If we leave all parameters to default values, we get {:.1f}".format(HCN_binding_energy_naive))
print("for the HCN binding energy.")

If we leave all parameters to default values, we get 4058.4 K
for the HCN binding energy.


# 2. Results for HCN binding energy

In [4]:
n_i_HCN = n_H2_at_Tjump * X_HCN
print("Number density of gas-phase HCN at HCN ice line: {:.1f} cm-3".format(n_i_HCN))

m_HCN = 1 + 12 + 14 # H + C + N
nu_HCN = 1.6e11 * np.sqrt(4024.7 * m_HCN)
print("vibrational frequency of HCN in the surface potential well: {:.2e} s-1".format(nu_HCN))

# thermal velocity.
v_HCN = (1.6 * np.sqrt(c.k_B * T_jump / (m_HCN*c.m_p))).to(u.cm/u.s)
print("Thermal velocity of HCN at {:.1f}: {:.1e}".format(T_jump, v_HCN))

HCN_binding_energy = binding_energy_given_desorption_temp(T_jump, n_i=n_i_HCN, nu_i=nu_HCN, v_i=v_HCN.value)
print("\nBinding energy of HCN: E_i/k = {:.1f}".format(HCN_binding_energy))

from IPython.display import display, Math

txt = "\\textrm{{Binding energy of HCN}}: \\boxed{{ \\frac{{E_i}}{{k}} = {:.1f}~\\textrm{{K}} }}".format(HCN_binding_energy.value)
display(Math(txt))


Number density of gas-phase HCN at HCN ice line: 4.9 cm-3
vibrational frequency of HCN in the surface potential well: 5.27e+13 s-1
Thermal velocity of HCN at 71.2 K: 2.4e+04 cm / s

Binding energy of HCN: E_i/k = 4002.4 K


<IPython.core.display.Math object>

In [5]:
# Now with errorbars

T_jump_high = T_jump + 2.4 * u.K
T_jump_low = T_jump - 2.6 * u.K

HCN_binding_energy_high = binding_energy_given_desorption_temp(T_jump_high, n_i=n_i_HCN, nu_i=nu_HCN, v_i=v_HCN.value)
HCN_binding_energy_low = binding_energy_given_desorption_temp(T_jump_low, n_i=n_i_HCN, nu_i=nu_HCN, v_i=v_HCN.value)

print(HCN_binding_energy_high)
print(HCN_binding_energy_low)
print("")
print("+", HCN_binding_energy_high-HCN_binding_energy)
print("-", HCN_binding_energy-HCN_binding_energy_low)


4137.361982505839 K
3856.291195650823 K

+ 134.9139776904076 K
- 146.15680916460815 K


## 2.1: Literature values for HCN binding energy

For reference, here are binding energies of other molecules (from Schwarz+14 and Hollenbach+2009, respectively):

![Table 1 from Schwarz et al. 2014][tableSchwarz]
![Table 1 from Hollenbach et al. 2009][tableHollenbach]

[tableHollenbach]: Hollenbach09_table1.png
[tableSchwarz]: Schwarz14_table1.png

Additionally,

2460 K -- 6974 K : Bertin et al 2017

3370 -- 3610 K : Noble et al 2013

1722 K : Bergin et al 1995

1760 K: Hasegawa & Herbst 1993 "New gas-grain chemical models of quiescent dense interstellar clouds - The effects of H2 tunnelling reactions and cosmic ray induced desorption"